In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
import string 
import re 
import tensorflow as tf
import numpy as np
import unicodedata
import pickle
from pickle import dump
from pickle import load
from numpy import array, argmax, random, take 
import pandas as pd 
import matplotlib.pyplot as plt 
% matplotlib inline 
pd.set_option('display.max_colwidth', 200)

In [3]:
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint 
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 
from keras import optimizers

Using TensorFlow backend.


In [0]:
# function to load a clean dataset
def load_data(filename):
	return load(open(filename, 'rb'))

In [0]:
# load English dataset i.e pickled earlier
filename = 'full_data.pkl'
data = load_data(filename)

In [6]:
data.head()

,eng,ger
0,Prague Stock Market falls to minus by the end of the trading day,Die Prager Börse stürzt gegen Geschäftsschluss ins Minus.
1,"After a sharp drop in the morning, the Prague Stock Market corrected its losses.",Nach dem steilen Abfall am Morgen konnte die Prager Börse die Verluste korrigieren.
2,Transactions with stocks from the Czech Energy Enterprise (ČEZ) reached nearly half of the regular daily trading.,Die Transaktionen mit den Aktien von ČEZ erreichten fast die Hälfte des normalen Tagesgeschäfts.
3,"The Prague Stock Market immediately continued its fall from Monday at the beginning of Tuesday's trading, when it dropped by nearly six percent.","Die Prager Börse knüpfte gleich zu Beginn der Dienstagsgeschäfte an den Einbruch vom Montag an, als sie um weitere sechs Prozentpunkte sank."
4,This time the fall in stocks on Wall Street is responsible for the drop.,Diesmal lag der Grund für den Einbruch an der Wall Street.


In [0]:
# taking random sample of 5000

data=data.sample(5000)


In [8]:
data.shape

(5000, 2)

In [0]:
subset = data[['eng', 'ger']]
raw_data = [tuple(x) for x in subset.values]

In [0]:
import unicodedata
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def normalize_string(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s

In [0]:
raw_data_en, raw_data_fr = list(zip(*raw_data))
raw_data_en, raw_data_fr = list(raw_data_en), list(raw_data_fr)

raw_data_en = [normalize_string(data) for data in raw_data_en]
raw_data_fr_in = ['<start> ' + normalize_string(data) for data in raw_data_fr]
raw_data_fr_out = [normalize_string(data) + ' <end>' for data in raw_data_fr]

# Seq2seq

In [0]:
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

In [0]:
en_tokenizer.fit_on_texts(raw_data_en)

In [18]:
print(en_tokenizer.word_index)

{'the': 1, '.': 2, 'of': 3, 'to': 4, 'in': 5, 'a': 6, 'and': 7, 'is': 8, 'that': 9, 'for': 10, 'on': 11, 'it': 12, 'with': 13, 'was': 14, 's': 15, 'as': 16, 'are': 17, 'be': 18, 'by': 19, 'at': 20, 'he': 21, 'not': 22, 'from': 23, 'this': 24, 'have': 25, 'has': 26, 'will': 27, 'his': 28, 'said': 29, 'they': 30, 'an': 31, 'which': 32, 'but': 33, 'i': 34, 'their': 35, 'who': 36, 'we': 37, 'there': 38, 'one': 39, 'were': 40, 'had': 41, 'its': 42, 'also': 43, 'been': 44, 'more': 45, 'or': 46, 'you': 47, 'all': 48, 'year': 49, 'about': 50, 'after': 51, 'would': 52, 'people': 53, 'new': 54, 'when': 55, 'can': 56, 'time': 57, 'only': 58, 'two': 59, 'no': 60, 'other': 61, 'than': 62, 'if': 63, 'so': 64, 't': 65, 'up': 66, 'out': 67, 'she': 68, 'years': 69, 'into': 70, 'first': 71, 'what': 72, 'some': 73, 'do': 74, 'her': 75, 'them': 76, 'could': 77, 'according': 78, 'state': 79, 'last': 80, 'even': 81, 'over': 82, 'now': 83, 'says': 84, 'these': 85, 'three': 86, 'because': 87, 'then': 88, '?':

In [0]:
data_en = en_tokenizer.texts_to_sequences(raw_data_en)

In [0]:

#padding
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en,
                                                        padding='post')

In [22]:
print(data_en[:1])

[[ 128  165   30   25    6  838    9  995    4  260  534 4563 4564 1961
     6 6762 1962    2    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

### Same for german data

In [0]:
fr_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

# ATTENTION: always finish with fit_on_texts before moving on
fr_tokenizer.fit_on_texts(raw_data_fr_in)
fr_tokenizer.fit_on_texts(raw_data_fr_out)

data_fr_in = fr_tokenizer.texts_to_sequences(raw_data_fr_in)
data_fr_in = tf.keras.preprocessing.sequence.pad_sequences(data_fr_in,
                                                           padding='post')

data_fr_out = fr_tokenizer.texts_to_sequences(raw_data_fr_out)
data_fr_out = tf.keras.preprocessing.sequence.pad_sequences(data_fr_out,
                                                            padding='post')

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(
    (data_en, data_fr_in, data_fr_out))
dataset = dataset.shuffle(20).batch(5)

# Seq2Seq model without Attention